<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Dataforst%C3%A5else-og-unders%C3%B8kelse-Pradeep/Bioco_Databehandling_av_manglende_verdier_og_utstikkere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige pakker og bibliotek

In [1]:
! pip install pyod

Importering av nødvendige pakker og bibliotek

In [2]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime

from pyod.models.hbos import HBOS

Importering av data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
bioco_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/førbehandlet_bioco_data.csv',
                         header=0,
                         sep=',',
                         index_col=0)

# Formatterer index til riktig format og datatype
bioco_data.index = pd.to_datetime(bioco_data.index,
                                  format='%Y-%m-%d %H:%M:%S')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Oppdeling av data til prediktorer- og responsvariabler

In [16]:
# Datasett for prediktorer
prediktor_data = bioco_data.iloc[:, :-4]
prediktorer = prediktor_data.columns

# Verdier for prediktorer
X = prediktor_data.values

# Datasett for hver kvalitetsmåling
collagen = bioco_data['Collagen']
mw = bioco_data['Mw']
smallmolecules = bioco_data['SmallMolecules']
brixadjusted = bioco_data['BrixAdjusted']

# Behandling av manglende verdier

Oversikt over antall manglende verider for per kolonne

In [5]:
bioco_data.isnull().sum()

EnzymeType_A1         0
EnzymeType_A2         0
EnzymeType_B          0
EnzymeType_C          0
EnzymeType_D          0
EnzymeType_E          0
RawMatFlow           55
WaterFlow            55
RawMatPercent        55
NIRfat            14187
NIRprotein        15731
NIRash            13976
NIRwater          13958
TT07                 50
TT08                 45
PT03                 25
TT20                 25
TT09                 15
TT12                  0
Collagen          43152
Mw                42711
SmallMolecules    42711
BrixAdjusted      42711
dtype: int64

Oversikt over rader med manglende verdier for RawMatflow

In [6]:
# Henter ut index til rader med manglende verdier for "RawMatFlow"
rawmat_manglende_index = bioco_data[bioco_data['RawMatFlow'].isnull()].index

# Sjekker hvilke andre kolonner som har manglende verdier for samme rader
bioco_data[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].isnull().sum()

# Dropper de spesifikke rader fra datasett
prosessert_data = bioco_data.drop(bioco_data.loc[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].index)

## Behandling av manglende verdier for NIR målinger

### Dropping av bestemte intervaller med manglende verdier

Oversikt over rader med manglende verdier for NIRfat

In [7]:
# Henter ut index til rader med manglende verdier for "NIRfat"
nirfat_manglende_index = prosessert_data[prosessert_data['NIRfat'].isnull()].index

# Lager liste for å lagre start og slutttid for serie med manglende verdier
nirfat_manglende_index_start = list()
nirfat_manglende_index_slutt = list()

# Legger til starttid for første serie med manglende verdier
nirfat_manglende_index_start.append(nirfat_manglende_index[0])

# Lagrer alle start og slutttid for serier med manglende verdier
for i in range(1, len(nirfat_manglende_index)):
  tidsdiff = nirfat_manglende_index[i] - nirfat_manglende_index[i-1]

  if tidsdiff.total_seconds() > 60:
    nirfat_manglende_index_slutt.append(nirfat_manglende_index[i-1])
    nirfat_manglende_index_start.append(nirfat_manglende_index[i])
nirfat_manglende_index_slutt.append(nirfat_manglende_index[-1])

# Maks grense på korte intervall for interpolering
diff_max = 15

# Lagring av antall manglende verider i korte intervaller
res_nir_manglende_korteintervall = 0

# Lager liste for start og slutt på korte intervaller med manglende verdier for NIRfat
nirfat_manglende_maks15 = list()

# Lager liste for start og slutt på lengre intervaller med manglende verdier for NIRfat
nirfat_manglende_min15 = list()


for i,j in zip(nirfat_manglende_index_start, nirfat_manglende_index_slutt):
  nirfat_mangel = prosessert_data[i:j].isnull().sum()['NIRfat']

# Lagrer alle start og sluttid for korte intervaller mindre enn maks tid
  if nirfat_mangel <= diff_max:
    nirfat_manglende_maks15.append((i,j))

    # Summerer antall manglende verdier i de korte intertaller
    res_nir_manglende_korteintervall += nirfat_mangel

    # Sjekker om de korte intervaller inneholder kvalitetsmålinger
    respons_mangel = prosessert_data[i:j].isnull().sum()[['Collagen',
                                                          'Mw',
                                                          'SmallMolecules',
                                                          'BrixAdjusted']]
    if nirfat_mangel > min(respons_mangel.values):
      print("Tidsintervallet {} til {} har flere manglende NIRfat målinger enn kvalitetsmålinger".format(i, j))
      print("NIRfat {}".format(nirfat_mangel))
      print(respons_mangel)

# Lagrer alle start og slutttid for lengre intervaller mer enn maks tid
  else:
    nirfat_manglende_min15.append((i,j))

print(res_nir_manglende_korteintervall,
      "av",
      prosessert_data.isnull().sum()['NIRfat'],
      "NIRfat målinger som mangler er i kortere intervaller enn",
      diff_max,
      "min")

132 av 14132 NIRfat målinger som mangler er i kortere intervaller enn 15 min


Dropper de intervaller mer enn 15 min med manglende verdier for NIRfat

In [8]:
for start, end in nirfat_manglende_min15:
    prosessert_data = prosessert_data[(prosessert_data.index < start) | (prosessert_data.index > end)]

EnzymeType_A1         0
EnzymeType_A2         0
EnzymeType_B          0
EnzymeType_C          0
EnzymeType_D          0
EnzymeType_E          0
RawMatFlow            0
WaterFlow             0
RawMatPercent         0
NIRfat              132
NIRprotein         1676
NIRash              178
NIRwater            204
TT07                  0
TT08                  0
PT03                  0
TT20                  0
TT09                  0
TT12                  0
Collagen          29107
Mw                28736
SmallMolecules    28736
BrixAdjusted      28736
dtype: int64

Oversikt over manglende verdier for NIRwater

In [11]:
# Henter ut index til rader med manglende verdier for "NIRwater"
nirwater_manglende_index = prosessert_data[prosessert_data['NIRwater'].isnull()].index

# Lager liste for å lagre start og slutttid for serie med manglende verdier
nirwater_manglende_index_start = list()
nirwater_manglende_index_slutt = list()

# Legger til starttid for første serie med manglende verdier
nirwater_manglende_index_start.append(nirwater_manglende_index[0])

# Lagrer alle start og slutttid for serier med manglende verdier
for i in range(1, len(nirwater_manglende_index)):
  tidsdiff = nirwater_manglende_index[i] - nirwater_manglende_index[i-1]

  if tidsdiff.total_seconds() > 60:
    nirwater_manglende_index_slutt.append(nirwater_manglende_index[i-1])
    nirwater_manglende_index_start.append(nirwater_manglende_index[i])
nirwater_manglende_index_slutt.append(nirwater_manglende_index[-1])

# Maks grense på korte intervall for interpolering
diff_max = 15

# Lagring av antall manglende verider i korte intervaller
res_water_manglende_korteintervall = 0

# Lager liste for start og slutt på korte intervaller med manglende verdier for NIRwater
nirwater_manglende_maks15 = list()

# Lager liste for start og slutt på lengre intervaller med manglende verdier for NIRwater
nirwater_manglende_min15 = list()


for i,j in zip(nirwater_manglende_index_start, nirwater_manglende_index_slutt):
  nirwater_mangel = prosessert_data[i:j].isnull().sum()['NIRwater']

# Lagrer alle start og sluttid for korte intervaller mindre enn maks tid
  if nirwater_mangel <= diff_max:
    nirwater_manglende_maks15.append((i,j))

    # Summerer antall manglende verdier i de korte intertaller
    res_water_manglende_korteintervall += nirwater_mangel

    # Sjekker om de korte intervaller inneholder kvalitetsmålinger
    respons_mangel = prosessert_data[i:j].isnull().sum()[['Collagen',
                                                          'Mw',
                                                          'SmallMolecules',
                                                          'BrixAdjusted']]
    if nirwater_mangel > min(respons_mangel.values):
      print("Tidsintervallet {} til {} har flere manglende NIRwater målinger enn kvalitetsmålinger".format(i, j))
      print("NIRwater {}".format(nirwater_mangel))
      print(respons_mangel)

# Lagrer alle start og slutttid for lengre intervaller mer enn maks tid
  else:
    nirwater_manglende_min15.append((i,j))

print(res_water_manglende_korteintervall,
      "av",
      prosessert_data.isnull().sum()['NIRwater'],
      "NIRwater målinger som mangler er i kortere intervaller enn",
      diff_max,
      "min")

144 av 204 NIRwater målinger som mangler er i kortere intervaller enn 15 min


Dropper de intervaller mer enn 15 min med manglende verdier for NIRwater

In [13]:
for start, end in nirwater_manglende_min15:
    prosessert_data = prosessert_data[(prosessert_data.index < start) | (prosessert_data.index > end)]

Oversikt over manglende verdier for NIRash

In [14]:
# Henter ut index til rader med manglende verdier for "NIRash"
nirash_manglende_index = prosessert_data[prosessert_data['NIRash'].isnull()].index

# Lager liste for å lagre start og slutttid for serie med manglende verdier
nirash_manglende_index_start = list()
nirash_manglende_index_slutt = list()

# Legger til starttid for første serie med manglende verdier
nirash_manglende_index_start.append(nirash_manglende_index[0])

# Lagrer alle start og slutttid for serier med manglende verdier
for i in range(1, len(nirash_manglende_index)):
  tidsdiff = nirash_manglende_index[i] - nirash_manglende_index[i-1]

  if tidsdiff.total_seconds() > 60:
    nirash_manglende_index_slutt.append(nirash_manglende_index[i-1])
    nirash_manglende_index_start.append(nirash_manglende_index[i])
nirash_manglende_index_slutt.append(nirash_manglende_index[-1])

# Maks grense på korte intervall for interpolering
diff_max = 15

# Lagring av antall manglende verider i korte intervaller
res_ash_manglende_korteintervall = 0

# Lager liste for start og slutt på korte intervaller med manglende verdier for NIRash
nirash_manglende_maks15 = list()

# Lager liste for start og slutt på lengre intervaller med manglende verdier for NIRash
nirash_manglende_min15 = list()


for i,j in zip(nirash_manglende_index_start, nirash_manglende_index_slutt):
  nirash_mangel = prosessert_data[i:j].isnull().sum()['NIRash']

# Lagrer alle start og sluttid for korte intervaller mindre enn maks tid
  if nirash_mangel <= diff_max:
    nirash_manglende_maks15.append((i,j))

    # Summerer antall manglende verdier i de korte intertaller
    res_water_manglende_korteintervall += nirash_mangel

    # Sjekker om de korte intervaller inneholder kvalitetsmålinger
    respons_mangel = prosessert_data[i:j].isnull().sum()[['Collagen',
                                                          'Mw',
                                                          'SmallMolecules',
                                                          'BrixAdjusted']]
    if nirash_mangel > min(respons_mangel.values):
      print("Tidsintervallet {} til {} har flere manglende NIRash målinger enn kvalitetsmålinger".format(i, j))
      print("NIRash {}".format(nirash_mangel))
      print(respons_mangel)

# Lagrer alle start og slutttid for lengre intervaller mer enn maks tid
  else:
    nirash_manglende_min15.append((i,j))

print(res_ash_manglende_korteintervall,
      "av",
      prosessert_data.isnull().sum()['NIRash'],
      "NIRash målinger som mangler er i kortere intervaller enn",
      diff_max,
      "min")

0 av 148 NIRash målinger som mangler er i kortere intervaller enn 15 min


## Interpolering av resterende manglende verdier

LOF for behandling av manglende verdier

In [17]:
for prediktor in prediktorer:
  prosessert_data[prediktor] = prosessert_data[prediktor].fillna(method = 'ffill')

# Inspeksjon av mulige ekstremverdier

## Ekstremverdi inspeksjon med HBOS

In [ ]:
hbos = HBOS()
hbos.fit(X)

y_predikert = hbos.labels_
y_scores = hbos.decision_scores_

ValueError: Input contains NaN.

Oppdeling av original datasett for hver kvalitetsmåling

In [ ]:
# Trekker ut prediktorer som et eget datasett
prediktor_data = bioco_rå_data.iloc[:, :-4]

# Datasett for "Collagen"
collagen_datasett = pd.concat([prediktor,
                               bioco_rå_data['Collagen']],
                              axis=1)

# Datasett for "Mw"
mw_datasett = pd.concat([prediktor,
                         bioco_rå_data['Mw']],
                        axis=1)

# Datasett for "SmallMolecules"
smallmolecules_datasett = pd.concat([prediktor,
                                     bioco_rå_data['SmallMolecules']],
                                    axis=1)

# Datasett for "BrixAdjusted"
brixadjusted_datasett = pd.concat([prediktor,
                                   bioco_rå_data['BrixAdjusted']],
                                  axis=1)

Lagrer hvert datasett som csv

In [ ]:
# Collagen datasett som csv fil
collagen_datasett.to_csv('')